<a href="https://colab.research.google.com/github/hellen2021/Football-Analysis-Mchezopesa-Limited/blob/main/Supervised_Learning_Regression_week1_IP_Core_Module2_15_04_2022_Hellen_Cheptoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Learning with Exploratory Data Analysis

### Football Analysis for Mchezopesa Limited Company

## 1.Defining the Research Questions

You have been recruited as a football analyst in a company - Mchezopesa Ltd and tasked to accomplish the task below.

A prediction result of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the game is friendly (include rank in your training).

## 2.Defining the metric of success
This project will be a success if:

a) We are able to accurately predict the winning team based on the two conditions:
*   Home team or away team
*   Type of tournament

b) Achieve an accuracy of atleast 80%

c) have the lowest RMSE as possible

## 3.Understanding the context

## 4.Experimental Design
The following are the design steps:
*   Loading the datsets
*   Exploring and cleaning the datasets
*   Merge the two datasets
*   Perform EDA
*   Feature Engineering
*   Check for multicollinearity
*   Polynomial Regression
*   Logistic Regression
*   Compute the Root Mean Squared Error
*   Create residual plots for the models, and assess their heteroscedasticity using Bartlett’s test

## 4.Data Relevance
The relevance of the data will be verified by the performance of the model



### Importing necessary libraries

In [84]:
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,KFold,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

import os
import warnings
import datetime

### load the data

In [85]:
# Mounting Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
# Accessing the working directory
os.chdir('/content/drive/MyDrive/Moringa Data Science/CORE MODULE 2/week1IP')

In [87]:
# Loading dataset
results = pd.read_csv('results.csv')
fifa_ranking = pd.read_csv('fifa_ranking.csv')

### Datasets Preview

In [88]:
# results dataset
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [89]:
# fifa ranking dataset
fifa_ranking.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


### Explore and clean the datasets

In [90]:
print('results dataset has',results.shape[0], 'rows and', results.shape[1], 'columns')
print('fifa ranking dataset has',fifa_ranking.shape[0], 'rows and', fifa_ranking.shape[1], 'columns')

results dataset has 40839 rows and 9 columns
fifa ranking dataset has 57793 rows and 16 columns


In [91]:
# shaping the column names
results.columns = results.columns.str.lower().str.strip().str.replace(' ', '_')
fifa_ranking.columns = fifa_ranking.columns.str.lower().str.strip().str.replace(' ', '_')

In [92]:
#renaming columns
fifa_ranking.rename(columns = {'country_full':'country'}, inplace = True)

In [93]:
# drop unnecessary columns
fifa_ranking.drop(['country_abrv'], axis = 1, inplace = True)
results.drop(['city'], axis = 1, inplace = True)

In [94]:
# statistical analysis
results.describe()

,home_score,away_score
count,40839.000000,40839.000000
mean,1.745709,1.188105
std,1.749145,1.405120
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


In [95]:
# statistical analysis
fifa_ranking.describe()

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted
count,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000,57793.000000
mean,101.628086,122.068637,332.302926,-0.009897,61.798602,61.798602,61.004602,30.502377,59.777462,17.933277,59.173916,11.834811
std,58.618424,260.426863,302.872948,5.804309,138.014883,138.014883,137.688204,68.844143,136.296079,40.888849,135.533343,27.106675
min,1.000000,0.000000,0.000000,-72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.000000,0.000000,56.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,101.000000,0.000000,272.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,152.000000,92.790000,525.000000,1.000000,32.250000,32.250000,26.660000,13.330000,21.500000,6.450000,21.250000,4.250000
max,209.000000,1775.030000,1920.000000,92.000000,1158.660000,1158.660000,1169.570000,584.790000,1159.710000,347.910000,1200.770000,240.150000


In [96]:
# info
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40839 entries, 0 to 40838
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        40839 non-null  object
 1   home_team   40839 non-null  object
 2   away_team   40839 non-null  object
 3   home_score  40839 non-null  int64 
 4   away_score  40839 non-null  int64 
 5   tournament  40839 non-null  object
 6   country     40839 non-null  object
 7   neutral     40839 non-null  bool  
dtypes: bool(1), int64(2), object(5)
memory usage: 2.2+ MB


date column data type should be changed to date time

In [97]:
# info
fifa_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   rank                     57793 non-null  int64  
 1   country                  57793 non-null  object 
 2   total_points             57793 non-null  float64
 3   previous_points          57793 non-null  int64  
 4   rank_change              57793 non-null  int64  
 5   cur_year_avg             57793 non-null  float64
 6   cur_year_avg_weighted    57793 non-null  float64
 7   last_year_avg            57793 non-null  float64
 8   last_year_avg_weighted   57793 non-null  float64
 9   two_year_ago_avg         57793 non-null  float64
 10  two_year_ago_weighted    57793 non-null  float64
 11  three_year_ago_avg       57793 non-null  float64
 12  three_year_ago_weighted  57793 non-null  float64
 13  confederation            57793 non-null  object 
 14  rank_date             

In [98]:
# missing values
results.isnull().sum()

# no missing values

date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
country       0
neutral       0
dtype: int64

In [99]:
# missing values
fifa_ranking.isnull().sum()

# there are no missing values

rank                       0
country                    0
total_points               0
previous_points            0
rank_change                0
cur_year_avg               0
cur_year_avg_weighted      0
last_year_avg              0
last_year_avg_weighted     0
two_year_ago_avg           0
two_year_ago_weighted      0
three_year_ago_avg         0
three_year_ago_weighted    0
confederation              0
rank_date                  0
dtype: int64

In [100]:
# duplicate values
value = results.duplicated().sum()
print(value)
print('results dataset has', value, 'duplicate values')
# results dataset has no duplicate values

0
results dataset has 0 duplicate values


In [101]:
# duplicate values
value = fifa_ranking.duplicated().sum()
print(value)
print('results dataset has', value, 'duplicate values')

37
results dataset has 37 duplicate values


In [102]:
# drop the duplicate values
fifa_ranking.drop_duplicates(keep='first', inplace = True)

### merge the two datasets

In [103]:
# first, change the date columns to datetime
results['date'] = pd.to_datetime(results['date'])

# rename the rank_date column
fifa_ranking.rename(columns = {'rank_date':'date'}, inplace = True)
fifa_ranking['date'] = pd.to_datetime(fifa_ranking['date'])   

In [104]:
# split the date to months and year
# results
results['year'] = results.date.dt.year
results['month'] = results.date.dt.month

# fifa rankings
fifa_ranking['year'] = fifa_ranking.date.dt.year
fifa_ranking['month'] = fifa_ranking.date.dt.month


Home_team

In [105]:
# merge to obtain the home_team

home_football = pd.merge(results, fifa_ranking, how = 'inner', left_on = ['year', 'month', 'home_team'], right_on = ['year', 'month', 'country'])

In [106]:
# preview the new dataframe
pd.set_option('display.max_columns', None)
home_football.head()

,date_x,home_team,away_team,home_score,away_score,tournament,country_x,neutral,year,month,rank,country_y,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,date_y
0,1993-08-01,Colombia,Paraguay,0,0,FIFA World Cup qualification,Colombia,False,1993,8,19,Colombia,0.0,36,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
1,1993-08-15,Colombia,Argentina,2,1,FIFA World Cup qualification,Colombia,False,1993,8,19,Colombia,0.0,36,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
2,1993-08-29,Colombia,Peru,4,0,FIFA World Cup qualification,Colombia,False,1993,8,19,Colombia,0.0,36,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
3,1993-08-01,Peru,Argentina,0,1,FIFA World Cup qualification,Peru,False,1993,8,70,Peru,0.0,16,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
4,1993-08-08,Peru,Colombia,0,1,FIFA World Cup qualification,Peru,False,1993,8,70,Peru,0.0,16,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


In [107]:
home_football.shape

(18593, 25)